In [ ]:
#from main import *
from tqdm import tqdm
import networkx as nx
from pyecharts.options.global_options import InitOpts
import scipy.io as sio
import h5py
import numpy as np
import pandas as pd
import math
import time
import matplotlib.pyplot as plt
import pyecharts
import random
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType, SymbolType, GeoType
from shapely.wkb import dumps as b_dumps, loads as b_loads
import os
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geopandas as gpd
import statsmodels.api as sm # recommended import according to the docs
import copy
from multiprocessing import  Process


'''Notice! motify the constellation parameters if needed'''


In [ ]:
RADIUS = 6371000
# constellation params
constell_num = 2
Constellation = [0]*constell_num
HEIGHT = [0]*constell_num
ELEVATION = [0]*constell_num
NUM_ORBITS = [0]*constell_num
NUM_SATS_PER_ORBIT = [0]*constell_num
ANUM = [0]*constell_num
CONSTELL_ID = 0
'''Notice, modify the constellation parameters here'''

Constellation[CONSTELL_ID] = "Starlink1" 
HEIGHT[CONSTELL_ID] = 550000
ELEVATION[CONSTELL_ID] = 25
NUM_ORBITS[CONSTELL_ID] = 72
NUM_SATS_PER_ORBIT[CONSTELL_ID] = 22
ANUM[CONSTELL_ID] = 2 # num of antenna 
# CONSTELL_ID = 0
# Constellation[CONSTELL_ID] = 'Kuiper1'
# HEIGHT[CONSTELL_ID] = 630000
# ELEVATION[CONSTELL_ID] = 20
# NUM_ORBITS[CONSTELL_ID] = 34
# NUM_SATS_PER_ORBIT[CONSTELL_ID] = 34
# ANUM[CONSTELL_ID] = 2

constell_id=0#starlink 1584
NUM_SATELLITES = NUM_ORBITS[constell_id] * NUM_SATS_PER_ORBIT[constell_id]
satStartId = [0]
orbit_num=NUM_ORBITS[CONSTELL_ID]#72
sat_per_orbit=NUM_SATS_PER_ORBIT[CONSTELL_ID]#22
sat_num=orbit_num*sat_per_orbit
fp = "../data/TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp"
data = gpd.read_file(fp)
def cir_to_car(lat, lng, h):
    return (RADIUS+h)*math.cos(math.radians(lat))*math.cos(math.radians(lng)),\
           (RADIUS+h)*math.cos(math.radians(lat))*math.sin(math.radians(lng)),\
           (RADIUS+h)*math.sin(math.radians(lat))

def dis_car(A, B):
    dis2 = (A[0]-B[0])*(A[0]-B[0])+(A[1]-B[1])*(A[1]-B[1])+(A[2]-B[2])*(A[2]-B[2])
    return math.sqrt(dis2)

def elevation_from_dis(adis, h_sat):
    if(h_sat >= adis):
        ael = 90
    else:
        cosel = (RADIUS**2 + adis**2 - (RADIUS+h_sat)**2) / (2*RADIUS*adis) 
        ael = math.degrees(math.acos(cosel)) - 90
    return ael
def get_arc(p0, p1, r=6371000+HEIGHT[CONSTELL_ID]):#550000): # p0=[lon,lat]
	z0 = r*np.sin(np.radians(p0[1]))
	x0 = r*np.cos(np.radians(p0[1]))*np.cos(np.radians(p0[0]))
	y0 = r*np.cos(np.radians(p0[1]))*np.sin(np.radians(p0[0]))
	z1 = r*np.sin(np.radians(p1[1]))
	x1 = r*np.cos(np.radians(p1[1]))*np.cos(np.radians(p1[0]))
	y1 = r*np.cos(np.radians(p1[1]))*np.sin(np.radians(p1[0]))
	theta = np.arccos(np.dot((x0,y0,z0),(x1,y1,z1))/(r*r))
	return theta * r
def cycleCompute(height):
    r = RADIUS + height
    G = 6.67428e-11
    M = 5.965e+24
    res = 4*math.pi*math.pi*r*r*r/G
    res = res/M
    res = math.sqrt(res)
    return res


In [ ]:
#calculate NLRP
def get_satno(idx):# idx->[0,1584)
    '''
    Note: for starlink idx in [0,1584]
    INPUT: <satno(idx)>
    OUTPUT: {plane,sat_no}
    '''
    plane=idx//sat_per_orbit+1 #[1,24]
    sat_no=(idx+1)%sat_per_orbit
    sat_no=sat_per_orbit if sat_no==0 else sat_no
    return [int(plane),int(sat_no)]# [1,24] [1,66]
def get_sid(p,n):
    idx=(p-1)*sat_per_orbit+(n-1)
    return idx
def get_direction(satPos0,t,sid,lasting=86399,constellation_name='Starlink1'):
    if t>=lasting:
        satPospre=load_satPos(f'/home/chenwei/CrossShellRouting/pos_data/{constellation_name}/{constellation_name}_{t-1}.txt')
        if satPospre[sid]['lat']<satPos0[sid]['lat']:
            return 1
        return 0
    else:
        satPos1=load_satPos(f'/home/chenwei/CrossShellRouting/pos_data/{constellation_name}/{constellation_name}_{t+1}.txt')
        if satPos0[sid]['lat']<satPos1[sid]['lat']:
            return 1#N is 1 South is 0
        return 0
def load_satPos(satfile):
    satPos=[]
    with open(satfile, 'r') as fr: # store all first
        lines = fr.readlines()
        for line in lines:
            lon,lat = float(line.split(',')[4]),float(line.split(',')[3])
            satPos.append({'lng':lon,'lat':lat})
    return satPos
def cal_nlrp(borders=[],large=1,d=1):
    l,r,u,b=borders[0]-large,borders[1]+large,-1,-1
    if l<=0:
        l=l+orbit_num
    if r>orbit_num:
        r=r-orbit_num
    if d==1:
        u=borders[2]+large
        b=borders[3]-large
        if u>sat_per_orbit:
            u=u-sat_per_orbit
        if b<=0:
            b=b+sat_per_orbit
            
    elif d==0:
        u=borders[2]-large
        b=borders[3]+large
        if u<=0:
            u=u+sat_per_orbit
        if b>sat_per_orbit:
            b=b-sat_per_orbit
    return [l,r,u,b]
    
def get_NLRP(ra=None,t=0,risk_country=['Egypt'],lasting=86400,constellation_name='Starlink1'):
    def write(nlrp=[],rsats=[],t=0,risk_country='Egypt'):
        file=f'/data/chenwei/research3/data/NLRP_24h/{constellation_name}/{risk_country}/{t}.txt'
        f=open(file,'w')
        f.write(f'{str(nlrp)}\n{str(rsats)}')
        f.close()
    satfile=f'/home/chenwei/CrossShellRouting/pos_data/{constellation_name}/{constellation_name}_{t}.txt'
    satPos=load_satPos(satfile)
    r_sats=[[-1 for _ in range(4)],[-1 for _ in range(4)]]
    nlrp=[[],[]]
    risk_sats_all=[set(),set()]
    for sid in range(sat_num):
        if ra.contains(Point(satPos[sid]['lng'],satPos[sid]['lat'])):
            d=get_direction(satPos,t,sid,lasting,constellation_name)
            if d==1 and r_sats[0][0]==-1:
                r_sats[0]=[sid,sid,sid,sid]
                risk_sats_all[0].add(sid)
            elif d==1:
                l,r,u,b=r_sats[0][0],r_sats[0][1],r_sats[0][2],r_sats[0][3]
                if satPos[l]['lng']>satPos[sid]['lng']:
                    r_sats[0][0]=sid
                if satPos[r]['lng']<satPos[sid]['lng']:
                    r_sats[0][1]=sid
                if satPos[u]['lat']<satPos[sid]['lat']:
                    r_sats[0][2]=sid
                if satPos[b]['lat']>satPos[sid]['lat']:
                    r_sats[0][3]=sid
                risk_sats_all[0].add(sid)
            elif d==0 and r_sats[1][0]==-1:
                r_sats[1]=[sid,sid,sid,sid]
                risk_sats_all[1].add(sid)
            elif d==0:
                l,r,u,b=r_sats[1][0],r_sats[1][1],r_sats[1][2],r_sats[1][3]
                if satPos[l]['lng']>satPos[sid]['lng']:
                    r_sats[1][0]=sid
                if satPos[r]['lng']<satPos[sid]['lng']:
                    r_sats[1][1]=sid
                if satPos[u]['lat']<satPos[sid]['lat']:
                    r_sats[1][2]=sid
                if satPos[b]['lat']>satPos[sid]['lat']:
                    r_sats[1][3]=sid
                risk_sats_all[1].add(sid)
    #NE-nlrp
    if risk_sats_all[0]!=set():
        l,r,u,b=r_sats[0][0],r_sats[0][1],r_sats[0][2],r_sats[0][3]
        pl,nl=get_satno(l)
        pr,nr=get_satno(r)
        pu,nu=get_satno(u)
        pb,nb=get_satno(b)
        for i in range(1,4):
            nlrp[0].append(cal_nlrp([pl,pr,nu,nb],i,1))
#SE-nlrp
    if risk_sats_all[1]!=set():
        l,r,u,b=r_sats[1][0],r_sats[1][1],r_sats[1][2],r_sats[1][3]
        pl,nl=get_satno(l)
        pr,nr=get_satno(r)
        pu,nu=get_satno(u)
        pb,nb=get_satno(b)    
        for i in range(1,4):
            nlrp[1].append(cal_nlrp([pl,pr,nu,nb],i,0))
    write(nlrp,risk_sats_all,t,risk_country)
    return nlrp,risk_sats_all
def run_nlrp(risk_country='Egypt',lasting=86400,step=600,constellation_name='Starlink1'):
    ra=data.loc[data['NAME'].str.contains(risk_country)].iloc[0]['geometry']
    def run(start,end,ra):
        with tqdm(list(range(start,end))) as T:
            for t in T:
                get_NLRP(ra,t,risk_country,lasting,constellation_name)
        T.close()
    cycle=lasting//step
    if lasting%step!=0:cycle+=1
    p=[]
    for _ in range(cycle):
        start=_*step
        end=start+step
        if end>=lasting:end=lasting
        p.append(Process(target=run,args=(start,end,ra)))
    for i in p:
        i.start()
    for i in p:
        i.join()
run_nlrp(risk_country='Egypt',lasting=6000-1,step=600,constellation_name='Kuiper1')
